In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import math

In [2]:
def get_request_element(requests, url):
    return requests.get(url)

In [3]:
def make_url(url):
    return f"https://www.property24.com{url}"

In [4]:
def save_url(list_url, url):
    if url not in list_url:
        list_url.append(url)
        
    return list_url

In [5]:
def def_url_page(page):
    return f"https://www.property24.com/for-sale/johannesburg/gauteng/100/p{page}"

In [6]:
def list_trap(soup):
    traps = []
    dot_list = []
    accolade_list = []
    container = soup.find('div', class_='p24_results')
    trap = str(container.find('style'))
    for pos, char in enumerate(trap):
        if char == '.':
            if pos not in dot_list:
                dot_list.append(pos)
        
        if char == '{':
            if pos not in accolade_list:
                accolade_list.append(pos)
                
    for i in range(len(accolade_list)):
        el = trap[dot_list[i]+1:accolade_list[i]]
        el = el[:el.find('\r')]
#         print(el)
        traps.append(el)
        
    return traps

In [7]:
begin_page = 1
end_page = 556

In [8]:
list_url_page = []

In [9]:
list_houses = []

### Recuperation de tous les infos cachés


In [10]:
def scrap_hidden_element(house, soup):
#     time.sleep(10)
    print('scrapp hidden element...')
    features = soup.find_all('div', class_='p24_propertyOverviewKey')
    features_value = soup.find_all('div', class_='p24_info')
    new_features = []
    for feature in features:
        new_features.append(feature.text.strip())
        if (feature.text.strip() == 'Street Address'):
            index = new_features.index('Street Address')
            features_value.insert(index, 'Pad d_addresse')
    
    length = len(features)
#     print('Len features = ', len(features))
#     print('len values = ', len(features_value))
    for i in range(length - 1):
        key = features[i].text.strip()
        if (type(features_value[i]) != str):
            value = features_value[i].text.strip().replace(' ', '')
            print(key + ' : ' + value)
    
        if (key == 'Rates and Taxes'):
            house[str(key)] = value
            
        elif (key == 'Levies'):
            house[str(key)] = value
            
        elif (key == 'Pets Allowed'):
            if (value == 'Yes'):
                house[str(key)] = 1
            elif(value == 'No'):
                house[str(key)] = 0
                
        elif (key == 'Erf Size'):
            house[str(key)] = value
            
        elif (key == 'Floor Size'):
            house[str(key)] = value
            
        elif (key == 'Type of Property'):
            house[str(key)] = value
            
    if ('Pool' in features or 'Pools' in features):
        print('Pool is in features')
        house['Pool'] = 1
    else:
        print('Pool is not in features')
        house['Pool'] = 0
        

### Recuperation de tous les elements visibles

In [11]:
def scrap_visible_element(house, soup):
    
#     print(soup)
#     time.sleep(5)
          
    if (soup.find('div', class_='p24_price') != None):
        prices = soup.find_all('div', class_='p24_price')
        if (len(prices) > 1):
#             print('First element price : ', prices[1])
            house['price'] = prices[1].text.strip()
#             print('Price : ', prices[1].text.strip().replace(" ", "").replace("R", ""))
        else:
#             print('First element price : ', prices[0])
            house['price'] = prices[0].text.strip()
#             print('Price : ', prices[0].text.strip().replace(" ", "").replace("R", ""))
    
    if (soup.find('li', title='Bedrooms') != None):
        house['bedrooms'] = soup.find('li', title='Bedrooms').text.strip()

    if (soup.find('li', title='Bathrooms') != None):
        house['bathrooms'] = soup.find('li', title='Bathrooms').text.strip()

    if (soup.find('li', title='Parking Spaces') != None):
        house['parking space'] = soup.find('li', title='Parking Spaces').text.strip()               


### Recuperation de tous les liens d' un element dans la liste de chaque page

In [12]:
print('Debut du recuperation de tous liens de chaque element')
while begin_page <= end_page:
    print('Page : ', begin_page)
    #definition de l' url du page courant
    url_page = def_url_page(begin_page)
    
    #faire une requete de la page
    response_page = get_request_element(requests, url_page)
    
    #attendre 5sec pour contourner le captcha
    time.sleep(5)
    
    # traitement de la page avec bs4
    soup_page = BeautifulSoup(response_page.text, "html.parser")
    
    #recuperation de tous les link contenant le lien de chaque element voulu avec repetition
    items = soup_page.find_all('div', class_='js_resultTile')
    traps = list_trap(soup_page)
    for item in items:
        value = []
        value = np.intersect1d(item['class'], traps)
        
        if (len(value) == 0):
            link = item.find('a').get('href')
            url = make_url(link)
            list_url_page = save_url(list_url_page, url)
            
        
    #On change de page
    begin_page += 1

print('Fin du recuperation de tous les liens de chaque element')

Debut du recuperation de tous liens de chaque element
Page :  535
Page :  536
Page :  537
Page :  538
Page :  539
Page :  540
Page :  541
Page :  542
Page :  543
Page :  544
Page :  545
Page :  546
Page :  547
Page :  548
Page :  549
Page :  550
Page :  551
Page :  552
Page :  553
Page :  554
Page :  555
Page :  556
Fin du recuperation de tous les liens de chaque element


### main 

In [13]:
print(list_url_page)

['https://www.property24.com/for-sale/haddon/johannesburg/gauteng/4318/109497798', 'https://www.property24.com/for-sale/killarney/johannesburg/gauteng/4388/108703423', 'https://www.property24.com/for-sale/lehae/johannesburg/gauteng/17396/109708283', 'https://www.property24.com/for-sale/yeoville/johannesburg/gauteng/5932/110683284', 'https://www.property24.com/for-sale/linksfield-north/johannesburg/gauteng/4359/110437288', 'https://www.property24.com/for-sale/west-turffontein/johannesburg/gauteng/4339/108602294', 'https://www.property24.com/for-sale/marshalltown/johannesburg/gauteng/3867/107651954', 'https://www.property24.com/for-sale/reuven/johannesburg/gauteng/5344/108317597', 'https://www.property24.com/for-sale/richmond/johannesburg/gauteng/16018/109572325', 'https://www.property24.com/for-sale/denver/johannesburg/gauteng/4204/110328891', 'https://www.property24.com/for-sale/newtown/johannesburg/gauteng/3869/108632105', 'https://www.property24.com/for-sale/houghton-estate/johannesb

In [14]:

for url in list_url_page:
    house = {}
    
    print('New url : ', url)
    
    #Faire une requete de la page
    res_page = get_request_element(requests, url)
    
    #attendre 5sec pour contourner le captcha
    time.sleep(5)
    
    # traitement de la page avec bs4
    soup = BeautifulSoup(res_page.text, "html.parser")
    
#     p24 = soup.find_all('div', class_='p24_info')
    #recuperation de tous les informations voulus
    scrap_visible_element(house, soup)
    scrap_hidden_element(house, soup)
    print(house)
    
    #ajout du nouvel house dans la liste des houses
    list_houses.append(house)
#     print('Lenght : ' ,len(list_houses))
#     df = pd.DataFrame(list_houses)
#     print(df.head())
print('Fin scrap')

New url :  https://www.property24.com/for-sale/haddon/johannesburg/gauteng/4318/109497798
scrapp hidden element...
Listing Number : 109497798
Type of Property : Apartment/Flat
Listing Date : 11January2021
Bedrooms : 2
Bathrooms : 1
Kitchens : 1
Lounges : 1
Pool is not in features
{'price': 'R 460 000', 'bedrooms': '2', 'bathrooms': '1', 'parking space': '1', 'Type of Property': 'Apartment/Flat', 'Pool': 0}
New url :  https://www.property24.com/for-sale/killarney/johannesburg/gauteng/4388/108703423
scrapp hidden element...
Listing Number : 108703423
Type of Property : Apartment/Flat
Listing Date : 03June2020
Bedrooms : 3
Bathrooms : 2
Kitchens : 1
Pool is not in features
{'price': 'R 2 250 000', 'bedrooms': '3', 'bathrooms': '2', 'parking space': '1', 'Type of Property': 'Apartment/Flat', 'Pool': 0}
New url :  https://www.property24.com/for-sale/lehae/johannesburg/gauteng/17396/109708283
scrapp hidden element...
Listing Number : 109708283
Type of Property : House
Description : NewDevelo

scrapp hidden element...
Listing Number : 110577892
Type of Property : CommercialProperty
Listing Date : 20October2021
Roller Shutter Doors : 1
Height of Roller Shutter Doors : 4m
Power Availability : ThreePhase
Security : GuardHouse,PerimeterWall
Pool is not in features
{'price': 'R 840 000', 'Type of Property': 'CommercialProperty', 'Pool': 0}
New url :  https://www.property24.com/for-sale/melrose-north/johannesburg/gauteng/12702/108858467
scrapp hidden element...
Listing Number : 108858467
Type of Property : Apartment/Flat
Listing Date : 10July2020
Floor Size : 52m²
Price per m² : R30673
Furnished : Yes
Bedroom : 1
Bathrooms : 1
Kitchens : 1
Office : 1
Parking : 1
Garden : Yes
Pool is not in features
{'price': 'R 1 595 000', 'bedrooms': '1', 'bathrooms': '1', 'parking space': '1', 'Type of Property': 'Apartment/Flat', 'Floor Size': '52m²', 'Pool': 0}
New url :  https://www.property24.com/new-developments-for-sale/rosebank/johannesburg/gauteng/the-tyrwhitt/5846/2806
scrapp hidden ele

scrapp hidden element...
Listing Number : 109488473
Type of Property : CommercialProperty
Listing Date : 08January2021
Erf Size : 1100m²
Pool is not in features
{'price': 'R 8 000 000', 'Type of Property': 'CommercialProperty', 'Erf Size': '1100m²', 'Pool': 0}
New url :  https://www.property24.com/for-sale/oakdene/johannesburg/gauteng/4323/109488460
scrapp hidden element...
Listing Number : 109488460
Type of Property : VacantLand/Plot
Listing Date : 08January2021
Pool is not in features
{'price': 'R 1 250 000', 'Type of Property': 'VacantLand/Plot', 'Pool': 0}
New url :  https://www.property24.com/for-sale/glenvista/johannesburg/gauteng/5317/109488457
scrapp hidden element...
Listing Number : 109488457
Type of Property : VacantLand/Plot
Listing Date : 08January2021
Pool is not in features
{'price': 'R 2 000 000', 'Type of Property': 'VacantLand/Plot', 'Pool': 0}
New url :  https://www.property24.com/for-sale/kew/johannesburg/gauteng/5281/109882940
scrapp hidden element...
Listing Numbe

scrapp hidden element...
Listing Number : 109972516
Type of Property : Apartment/Flat
Listing Date : 13May2021
Levies : R1800
Bedrooms : 2
Bathrooms : 1
Kitchens : 1
Pool is not in features
{'price': 'R 370 000', 'bedrooms': '2', 'bathrooms': '1', 'parking space': '1', 'Type of Property': 'Apartment/Flat', 'Levies': 'R1800', 'Pool': 0}
New url :  https://www.property24.com/for-sale/marshalltown/johannesburg/gauteng/3867/108322722
scrapp hidden element...
Listing Number : 108322722
Type of Property : Apartment/Flat
Listing Date : 15January2020
Floor Size : 43m²
Price per m² : R11628
Bedroom : 1
Pool is not in features
{'price': 'R 500 000', 'bedrooms': '1', 'bathrooms': '2', 'Type of Property': 'Apartment/Flat', 'Floor Size': '43m²', 'Pool': 0}
New url :  https://www.property24.com/for-sale/ridgeway/johannesburg/gauteng/5345/109167913
scrapp hidden element...
Listing Number : 109167913
Type of Property : Townhouse
Listing Date : 29September2020
Bedrooms : 2
Bathrooms : 1
Pool is not in 

scrapp hidden element...
Listing Number : 110072618
Type of Property : House
Listing Date : 09June2021
Erf Size : 2152m²
Rates and Taxes : R49228
Bathrooms : 1
Kitchens : 1
Office/study : Orflatlet
Domestic Rooms : 1
Domestic Bathrooms : 1
Garage : 3
Parking : SecureParking
Outbuilding : StaffQuarters/DomesticRooms,DomesticBathroom
Pool is not in features
{'price': 'R 1 699 000', 'bathrooms': '1', 'parking space': '4', 'Type of Property': 'House', 'Erf Size': '2152m²', 'Rates and Taxes': 'R49228', 'Pool': 0}
New url :  https://www.property24.com/for-sale/rispark/johannesburg/gauteng/5203/107755821
scrapp hidden element...
Listing Number : 107755821
Type of Property : House
Listing Date : 26July2019
Erf Size : 32375m²
Bedrooms : 4
Bedroom 1 : Bedroom4
Bedroom 2 : Bedroom3
Bedroom 3 : Bedroom2
Bedroom 4 : Bedroom1
Bathrooms : 2
Bathroom 1 : Bathroom2
Bathroom 2 : Bathroom1
Kitchen : Kitchen
Reception Rooms : 1
Lounges : 2
Lounge 1 : Lounge2
Lounge 2 : Lounge1
Dining Room : DiningRoom
Fam

...

scrapp hidden element...
Listing Number : 110392939
Type of Property : House
Listing Date : 06September2021
Erf Size : 200m²
Bedrooms : 1
Bathrooms : 1
Kitchens : 1
Pool is not in features
{'price': 'R 350 000', 'bedrooms': '1', 'bathrooms': '1', 'Type of Property': 'House', 'Erf Size': '200m²', 'Pool': 0}
New url :  https://www.property24.com/new-developments-for-sale/craighall/johannesburg/gauteng/featherstone/5820/3295
scrapp hidden element...
Pool is not in features
{'price': 'R 1 520 000 - R 1 755 000', 'Pool': 0}
New url :  https://www.property24.com/for-sale/lenasia-south/johannesburg/gauteng/3510/109268092
scrapp hidden element...
Listing Number : 109268092
Type of Property : VacantLand/Plot
Listing Date : 23October2020
Pool is not in features
{'price': 'R 1 650 000', 'Type of Property': 'VacantLand/Plot', 'Pool': 0}
New url :  https://www.property24.com/for-sale/lehae/johannesburg/gauteng/17396/107488796
scrapp hidden element...
Listing Number : 107488796
Type of Property : Ho

scrapp hidden element...
Listing Number : 107187790
Type of Property : VacantLand/Plot
Listing Date : 11February2019
Pool is not in features
{'price': 'R 4 500 000', 'Type of Property': 'VacantLand/Plot', 'Pool': 0}
New url :  https://www.property24.com/new-developments-for-sale/waverley/johannesburg/gauteng/36-on-knox/12704/3489
scrapp hidden element...
Pool is not in features
{'price': 'R 3 999 000 - R 4 249 000', 'Pool': 0}
New url :  https://www.property24.com/for-sale/lehae/johannesburg/gauteng/17396/109676628
scrapp hidden element...
Listing Number : 109676628
Type of Property : House
Listing Date : 24February2021
Bedrooms : 3
Bathrooms : 1
Kitchen : GraniteTops
Pool is not in features
{'price': 'R 747 000', 'bedrooms': '3', 'bathrooms': '1', 'Type of Property': 'House', 'Pool': 0}
New url :  https://www.property24.com/for-sale/glenvista/johannesburg/gauteng/5317/110168848
scrapp hidden element...
Listing Number : 110168848
Type of Property : VacantLand/Plot
Pool is not in featur

scrapp hidden element...
Listing Number : 109966543
Type of Property : House
Listing Date : 12May2021
Bedrooms : 3
Bathrooms : 1
Kitchens : 1
Lounges : 1
Dining Rooms : 1
Pool is not in features
{'price': 'R 980 000', 'bedrooms': '3', 'bathrooms': '1', 'Type of Property': 'House', 'Pool': 0}
New url :  https://www.property24.com/for-sale/bezuidenhout-valley/johannesburg/gauteng/4201/107152070
scrapp hidden element...
Listing Number : 107152070
Type of Property : CommercialProperty
Description : Retail,Storage,Warehouse
Listing Date : 27August2020
Gross Price : R35700
Net Price : R31000
Availability Date : 20September2021
Erf Size : 495m²
Building Name : VintageVault
Security : SecurityGate,BurglarBars,Safe
Pool is not in features
{'price': 'R 1 780 999', 'Type of Property': 'CommercialProperty', 'Erf Size': '495m²', 'Pool': 0}
New url :  https://www.property24.com/for-sale/turffontein/johannesburg/gauteng/4337/108760720
scrapp hidden element...
Listing Number : 108760720
Type of Proper

scrapp hidden element...
Listing Number : 105067199
Type of Property : CommercialProperty
Description : Office
Listing Date : 01October2019
Availability Date : 15February2021
Pool is not in features
{'price': 'R 4 000 000', 'Type of Property': 'CommercialProperty', 'Pool': 0}
New url :  https://www.property24.com/for-sale/hillbrow/johannesburg/gauteng/3862/109558593
scrapp hidden element...
Listing Number : 109558593
Type of Property : CommercialProperty
Listing Date : 26January2021
Pool is not in features
{'price': 'R 15 950 000', 'Type of Property': 'CommercialProperty', 'Pool': 0}
New url :  https://www.property24.com/new-developments-for-sale/rosebank/johannesburg/gauteng/the-tyrwhitt/5846/524
scrapp hidden element...
Pool is not in features
{'price': 'R 2 370 500 - R 2 552 550', 'Pool': 0}
New url :  https://www.property24.com/for-sale/west-turffontein/johannesburg/gauteng/4339/110012941
scrapp hidden element...
Listing Number : 110012941
Type of Property : Apartment/Flat
Listing 

scrapp hidden element...
Listing Number : 110874625
Type of Property : House
Listing Date : 25January2022
Pool is not in features
{'price': 'R 735 000', 'Type of Property': 'House', 'Pool': 0}
New url :  https://www.property24.com/for-sale/fonteine-ah/johannesburg/gauteng/32968/110772651
scrapp hidden element...
Listing Number : 110772651
Type of Property : VacantLand/Plot
Listing Date : 14December2021
Pool is not in features
{'price': 'R 2 800 000', 'Type of Property': 'VacantLand/Plot', 'Pool': 0}
New url :  https://www.property24.com/for-sale/forest-town/johannesburg/gauteng/4385/110699878
scrapp hidden element...
Listing Number : 110699878
Type of Property : Apartment/Flat
Description : SingleStorey,Penthouse
Listing Date : 21November2021
Erf Size : 2028m²
Floor Size : 233m²
Price per m² : R24464
Levies : R8605
Rates and Taxes : R1824
Pets Allowed : Yes
Bedrooms : 2
Bathrooms : 2.5
Kitchen : GuestToilet,Ensuite
Reception Rooms : 1
Lounge : 1
Dining Room : 1
Family/TV Room : 1
Entra

In [15]:
print(list_houses)

[{'price': 'R 460 000', 'bedrooms': '2', 'bathrooms': '1', 'parking space': '1', 'Type of Property': 'Apartment/Flat', 'Pool': 0}, {'price': 'R 2 250 000', 'bedrooms': '3', 'bathrooms': '2', 'parking space': '1', 'Type of Property': 'Apartment/Flat', 'Pool': 0}, {'price': 'R 559 000', 'bedrooms': '2', 'bathrooms': '1', 'Type of Property': 'House', 'Erf Size': '195m²', 'Floor Size': '40m²', 'Pool': 0}, {'price': 'R 3 990 000', 'Type of Property': 'House', 'Pool': 0}, {'price': 'R 2 950 000', 'Type of Property': 'VacantLand/Plot', 'Erf Size': '977m²', 'Pool': 0}, {'price': 'R 590 000', 'bedrooms': '2', 'bathrooms': '1', 'parking space': '1', 'Type of Property': 'Townhouse', 'Pets Allowed': 1, 'Pool': 0}, {'price': 'R 695 000', 'bedrooms': '2', 'bathrooms': '2', 'parking space': '1', 'Type of Property': 'Apartment/Flat', 'Floor Size': '84m²', 'Levies': 'R1700', 'Rates and Taxes': 'R400', 'Pool': 0}, {'price': 'R 1 200 000', 'bedrooms': '3', 'bathrooms': '1', 'Type of Property': 'House', '

In [16]:
df = pd.DataFrame(list_houses)

In [19]:
df

,price,bedrooms,bathrooms,parking space,Type of Property,Pool,Erf Size,Floor Size,Pets Allowed,Levies,Rates and Taxes
0,R 460 000,2,1,1,Apartment/Flat,0,NaN,NaN,NaN,NaN,NaN
1,R 2 250 000,3,2,1,Apartment/Flat,0,NaN,NaN,NaN,NaN,NaN
2,R 559 000,2,1,NaN,House,0,195m²,40m²,NaN,NaN,NaN
3,R 3 990 000,NaN,NaN,NaN,House,0,NaN,NaN,NaN,NaN,NaN
4,R 2 950 000,NaN,NaN,NaN,VacantLand/Plot,0,977m²,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
411,R 900 000,NaN,NaN,NaN,VacantLand/Plot,0,NaN,NaN,NaN,NaN,NaN
412,R 1 600 000,3,2,1,House,0,NaN,NaN,NaN,R800,R450
413,R 34 999 000,NaN,NaN,NaN,CommercialProperty,0,7449m²,NaN,NaN,NaN,NaN
414,R 21 999 000,NaN,NaN,NaN,CommercialProperty,0,3566m²,NaN,NaN,NaN,NaN


In [18]:
df.to_csv('Johannesburg.csv', index=False, encoding='utf-8')